In [1]:
# https://www.tensorflow.org/api_docs/python/tf/nn/bidirectional_dynamic_rnn

In [2]:
import tensorflow as tf

In [3]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

Using TensorFlow backend.


Exception: URL fetch failure on https://s3.amazonaws.com/text-datasets/imdb.npz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)

In [ ]:
class TextRNN(object):
    def __init__(self, args):
        self.args = args
        self.input_x = tf.placeholder(tf.int32, [None, self.args.seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, self.args.num_classes], name='input_y')
        self.global_step = tf.placeholder(shape=(), dtype=tf.int32, name='global_step')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        self.rnn()

    def _auc_pr(self, true, prob, depth=18):
        pred = tf.one_hot(tf.argmax(prob,-1), depth = depth)
        tp = tf.logical_and(tf.cast(pred, tf.bool), tf.cast(true, tf.bool))
        acc = tf.truediv(tf.reduce_sum(tf.cast(tp, tf.int32)), tf.shape(pred)[0])
        print(acc)

        return acc

    def rnn(self):
        # embedding lookup
        with tf.device('/cpu:0'):
            self.embedding = tf.Variable(
                tf.constant(0.0, shape=[self.args.vocab_size, self.args.embedding_dim]), trainable=True,
                name="embedding")
            self.embedding_placeholder = tf.placeholder(tf.float32,
                                                        [self.args.vocab_size, self.args.embedding_dim])
            self.embedding_init = self.embedding.assign(self.embedding_placeholder)
            self.embedding_inputs = tf.nn.embedding_lookup(self.embedding, self.input_x)
        # rnn layer after embedding
        with tf.name_scope('rnn-size-%s' % self.args.hidden_dim):
            lstm_cell = tf.nn.rnn_cell.LSTMCell(self.args.hidden_dim)
            outputs, states = tf.nn.bidirectional_dynamic_rnn(lstm_cell, lstm_cell, self.embedding_inputs, dtype=tf.float32)
            rnn_outputs = tf.concat((states[0][1], states[1][1]), 1)
            # https://www.cnblogs.com/gaofighting/p/9673338.html

        with tf.name_scope("score"):
            """ dense layer 1 """
            fc = tf.layers.dense(rnn_outputs, self.args.hidden_dim, name='fc1')
            fc = tf.nn.relu(fc)
            fc_1 = tf.nn.dropout(fc, self.keep_prob)
            """ dense layer 2 """
            result_dense = tf.layers.dense(fc_1, self.args.num_classes, name='fc2')
            self.result_softmax = tf.nn.softmax(result_dense, name='my_output')
            self.y_pred_cls = tf.argmax(self.result_softmax, 1, name='predict')  # 预测类别

        with tf.name_scope("optimize"):
            self.loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.result_softmax, labels=self.input_y)
            self.learning_rate = tf.train.exponential_decay(learning_rate=self.args.learning_rate,
                                                            global_step=self.global_step,
                                                            decay_steps=2,
                                                            decay_rate=0.95,
                                                            staircase=True)
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.trainer = optimizer.minimize(self.loss)
            tf.summary.scalar('loss', self.loss)
        
        with tf.name_scope("accuracy"):
            self.acc = self._auc_pr(self.input_y, self.result_softmax)

In [ ]:

    lstm_cell = tf.nn.rnn_cell.LSTMCell(self.args.hidden_dim)
    outputs, states = tf.nn.bidirectional_dynamic_rnn(lstm_cell, lstm_cell, self.embedding_inputs, dtype=tf.float32)
    rnn_outputs = tf.concat((states[0][1], states[1][1]), 1)
    # https://www.cnblogs.com/gaofighting/p/9673338.html

with tf.name_scope("score"):
    """ dense layer 1"""
    fc = tf.layers.dense(rnn_outputs, self.args.hidden_dim, name='fc1')
    fc = tf.nn.relu(fc)
    fc_1 = tf.nn.dropout(fc, self.keep_prob)
    """ dense layer 2"""
    result_dense = tf.layers.dense(fc_1, self.args.num_classes, name='fc2')
    self.result_softmax = tf.nn.softmax(result_dense, name='my_output')
    self.y_pred_cls = tf.argmax(self.result_softmax, 1, name='predict')  # 预测类别

with tf.name_scope("optimize"):
    self.loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.result_softmax, labels=self.input_y)
    self.learning_rate = tf.train.exponential_decay(learning_rate=self.args.learning_rate,
                                                    global_step=self.global_step,
                                                    decay_steps=2,
                                                    decay_rate=0.95,
                                                    staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
    self.trainer = optimizer.minimize(self.loss)
    tf.summary.scalar('loss', self.loss)

with tf.name_scope("accuracy"):
    self.acc = self._auc_pr(self.input_y, self.result_softmax)

In [ ]:
def train_with_embedding(model, args):
    step = 0

    categories, id_to_cat, cat_to_id = read_label_from_file(args.label_dir)
    word_to_id, embedding_matrix = read_vocab_and_embedding_from_pickle_file(args.vocab_embedding_file)

    print("Loading training and validation data...")
    start_time = time.time()
    x_train, y_train = get_encoded_texts_and_labels(args.train_file, word_to_id, cat_to_id, args.seq_length)

    from sklearn.model_selection import train_test_split

    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=0)

    time_dif = get_time_dif(start_time)
    print("Time usage for load data : ", time_dif)
    print("Session init...")
    session = tf.Session(config=args.gpu_settings)
    session.run(tf.global_variables_initializer())

    print("Load embedding...")
    session.run(model.embedding_init, feed_dict={model.embedding_placeholder: embedding_matrix})

    writer = tf.summary.FileWriter(args.save_dir)
    writer.add_graph(session.graph)

    print('Training and evaluating...')
    start_time = time.time()
    max_acc = 0
    last_improved = 0  # record lat improved epoch

    saver = tf.train.Saver()

    for epoch in range(args.num_epochs):
        print('\nEpoch:', epoch + 1)
        args.epoch = epoch

        batch_train = batch_iter_x_y(x_train, y_train, args.batch_size)

        """ batch data input"""
        for x_batch, y_batch in batch_train:
            feed_dict = feed_data(model, x_batch, y_batch, step, args.dropout_keep_prob)
            _, loss_train = session.run([model.trainer, model.loss], feed_dict=feed_dict)

        acc_val = evaluate(session, model, x_val, y_val, args.batch_size, step, 1.0)
        print('current accuracy on validation : ', acc_val)
        print('max accuracy on validation : ', max_acc)

        if acc_val > max_acc:
            step += 1
            max_acc = acc_val
            print("current max acc is " + str(max_acc))
            last_improved = epoch
            saver.save(sess=session, save_path=args.save_path)
            # proto
            output_graph_def = convert_variables_to_constants(session, session.graph_def, output_node_names=['score/my_output'])
            tf.train.write_graph(output_graph_def, args.export_dir, args.model_name, as_text=False)

            print("Learning_rate", session.run(model.learning_rate, feed_dict={model.global_step: epoch}))

        if epoch - last_improved > args.early_stopping_epoch:
            print("No optimization for a long time, auto-stopping...")
            time_dif = get_time_dif(start_time)
            print("Test time usage is : " + str(time_dif))
            break

    time_dif = get_time_dif(start_time)
    print("Train time usage is : " + str(time_dif))